# Pre processamento para criação do db mesclado de dialise e FAV

In [3]:
import pandas as pd

## carregando db de dialise

In [4]:
df_dialise = pd.read_csv("ATDRS_DIALISE.csv", sep=";", dtype=str)

## adicionando coluna "Cronico" para saber se a pessoa é cronica, aguda, ou se é um cronico repetido

In [5]:
# Converter colunas de datas
df_dialise['AP_CMP'] = pd.to_datetime(df_dialise['AP_CMP'] + '01', format='%Y%m%d')  # AP_CMP é AAAAMM
df_dialise['ATD_DTPDR'] = pd.to_datetime(df_dialise['ATD_DTPDR'], format='%Y%m%d', errors='coerce')

# Calcular diferença em meses
df_dialise['meses_dialise'] = (df_dialise['AP_CMP'].dt.year - df_dialise['ATD_DTPDR'].dt.year) * 12 + (df_dialise['AP_CMP'].dt.month - df_dialise['ATD_DTPDR'].dt.month)

# Classificação inicial
def classificar_cronico(meses):
    if pd.isna(meses):
        return 'agudo'  # ou 'desconhecido' se quiser outra label
    elif meses < 3:
        return 'agudo'
    else:
        return 'cronico'

df_dialise['Cronico'] = df_dialise['meses_dialise'].apply(classificar_cronico)

# Agora identificar quem já foi "cronico" em registros anteriores
df_dialise['Cronico_previo'] = df_dialise.groupby('AP_CNSPCN')['Cronico'].transform(lambda x: x.eq('cronico').cumsum().shift().fillna(0) > 0)
df_dialise.loc[df_dialise['Cronico_previo'] & (df_dialise['Cronico'] == 'cronico'), 'Cronico'] = 'repetido'

# Limpar auxiliar
df_dialise.drop(columns=['Cronico_previo'], inplace=True)
df_dialise.head()

,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,ATD_ALBUMI,ATD_PTH,ATD_HIV,ATD_HCV,ATD_HBSAG,ATD_INTERC,ATD_SEPERI,AP_NATJUR,meses_dialise,Cronico
0,201408,MN,431490,2237253,4314203219370,2014-08-01,0305010166,2149.62,431490,05,...,4,0,N,N,N,I,I,NaN,10.0,cronico
1,201408,MN,431490,2262568,4314203319756,2014-08-01,0305010166,400.0,431490,05,...,0,0,N,N,N,S,I,NaN,0.0,agudo
2,201408,MN,431490,2262568,4314203240490,2014-08-01,0305010166,2149.62,431490,05,...,0,0,N,N,N,N,N,NaN,70.0,cronico
3,201408,MN,431490,2237601,4314201305645,2014-08-01,0305010166,2204.49,431490,05,...,0004,0368,N,N,N,I,I,NaN,51.0,cronico
4,201408,MN,431490,2237598,4314203319162,2014-08-01,0305010166,1529.46,431490,05,...,4,534,N,P,N,S,N,NaN,52.0,cronico


## carregando db FAV


In [6]:
# 1. Carregar os dados fav
df_fav = pd.read_csv("ACFRS_FAV.csv", sep=";", dtype=str)


In [7]:
print(df_dialise[['AP_CMP', 'ATD_DTPDR', 'Cronico']].head(5))

      AP_CMP  ATD_DTPDR  Cronico
0 2014-08-01 2013-10-02  cronico
1 2014-08-01 2014-08-11    agudo
2 2014-08-01 2008-10-01  cronico
3 2014-08-01 2010-05-15  cronico
4 2014-08-01 2010-04-30  cronico


In [8]:
print(df_fav[['AP_CMP','AP_MVM']].head(5))

   AP_CMP  AP_MVM
0  201407  201408
1  201408  201408
2  201408  201408
3  201408  201408
4  201408  201408


## convertendo datas string em datetime

In [9]:
# 2. Converter datas importantes
df_dialise['AP_CMP'] = pd.to_datetime(df_dialise['AP_CMP'], format='%Y-%m-%d')
df_dialise['ATD_DTPDR'] = pd.to_datetime(df_dialise['ATD_DTPDR'], format='%Y-%m-%d', errors='coerce')

In [10]:
print(df_dialise[[ 'AP_CMP', 'ATD_DTPDR', 'Cronico']].head(5))

      AP_CMP  ATD_DTPDR  Cronico
0 2014-08-01 2013-10-02  cronico
1 2014-08-01 2014-08-11    agudo
2 2014-08-01 2008-10-01  cronico
3 2014-08-01 2010-05-15  cronico
4 2014-08-01 2010-04-30  cronico


## adicionando "fav_" no inicio de cada nome das colunas do db FAV
 

In [11]:
# 3. Renomear colunas do FAV (exceto a chave)
colunas_fav = [col for col in df_fav.columns if col != "AP_CNSPCN"]
df_fav_renomeado = df_fav.rename(columns={col: f"fav_{col}" for col in colunas_fav})


In [12]:
print(df_fav_renomeado[['fav_AP_CMP','fav_AP_MVM']].head(5))

  fav_AP_CMP fav_AP_MVM
0     201407     201408
1     201408     201408
2     201408     201408
3     201408     201408
4     201408     201408


In [13]:
df_fav_renomeado['fav_AP_CMP'] = pd.to_datetime(df_fav_renomeado['fav_AP_CMP'], format='%Y%m', errors='coerce')

In [14]:
print(df_fav_renomeado[['fav_AP_CMP','fav_AP_MVM']].head(5))

  fav_AP_CMP fav_AP_MVM
0 2014-07-01     201408
1 2014-08-01     201408
2 2014-08-01     201408
3 2014-08-01     201408
4 2014-08-01     201408


In [15]:
df_fav_teste = df_fav_renomeado.copy()

# print quantidade de AP_CNSPCN repetidos
print("Quantidade de AP_CNSPCN repetidos:" + str(df_fav_renomeado['AP_CNSPCN'].duplicated().sum()))
# print quantidade de AP_CNSPCN nulos
print("Quantidade de AP_CNSPCN nulos:" + str(df_fav_renomeado['AP_CNSPCN'].isnull().sum()))

# remover AP_CNSPCN repetidos
df_fav_renomeado = df_fav_renomeado.drop_duplicates(subset=['AP_CNSPCN'])
df_fav_renomeado = df_fav_renomeado.dropna(subset=['AP_CNSPCN'])

print("Quantidade de AP_CNSPCN repetidos:" + str(df_fav_renomeado['AP_CNSPCN'].duplicated().sum()))
print("Quantidade de AP_CNSPCN nulos:" + str(df_fav_renomeado['AP_CNSPCN'].isnull().sum()))




Quantidade de AP_CNSPCN repetidos:7417


Quantidade de AP_CNSPCN nulos:28
Quantidade de AP_CNSPCN repetidos:0
Quantidade de AP_CNSPCN nulos:0


In [16]:
# Conjuntos de CNS únicos
cns_fav = set(df_fav_renomeado['AP_CNSPCN'].unique())
cns_dialise = set(df_dialise['AP_CNSPCN'].unique())

# CNS do FAV que não aparecem na diálise
cns_fav_nao_encontrados = cns_fav - cns_dialise

print(f"Quantidade de AP_CNSPCN únicos no FAV que não aparecem na diálise: {len(cns_fav_nao_encontrados)}")
#printar alguns exemplos
print("Exemplos:", list(cns_fav_nao_encontrados)[:10])

Quantidade de AP_CNSPCN únicos no FAV que não aparecem na diálise: 921
Exemplos: ['\x82{\x7f\x80{{~||}|\x82\x81|\x81', '|}~\x84\x7f\x82~\x81\x82|\x7f{{{\x83', '\x82{{\x83{\x82\x7f\x80||\x80\x83~\x83\x84', '}{~\x84\x80\x82\x84~{\x7f\x81{{{|', '|\x83{{{|\x81\x83~\x83\x84{{{\x80', '\x82{~{{|\x83~\x84~\x82\x83}\x82{', '\x83\x84\x83{{~\x82|{\x80\x81|\x84\x83~', '\x82{\x83~{\x82}~||\x7f~\x80\x81{', '\x82{\x84\x81{\x81\x81\x82|~\x81{\x80\x82}', '\x83\x84\x83{{{{\x7f|\x82\x7f\x7f}\x84\x7f']


## Merge de dialise e FAV


In [17]:
# 4. Juntar os dados mantendo TODOS os registros de diálise
df_merged = pd.merge(df_dialise, 
                    df_fav_renomeado, 
                    on="AP_CNSPCN", 
                    how="left")


In [18]:
print(df_merged[['fav_AP_CMP','fav_AP_MVM']].head(5))

  fav_AP_CMP fav_AP_MVM
0 2016-11-01     201611
1        NaT        NaN
2        NaT        NaN
3        NaT        NaN
4        NaT        NaN


## Repetição de dialises depois da FAV

In [19]:
# 5. Adicionar coluna de repetição de FAV apenas para registros válidos
df_merged['fav_repeticao'] = df_merged.groupby('AP_CNSPCN')['fav_AP_CMP'].transform(lambda x: x.notna().cumsum())

# print fav_repeticao > 1
print("Quantidade de fav_repeticao > 1: " + str((df_merged['fav_repeticao'] > 1).sum()))

# print head 10 fav_repeticao > 1
print(df_merged[df_merged['fav_repeticao'] > 1].head(10))


Quantidade de fav_repeticao > 1: 419399
      AP_MVM AP_CONDIC AP_GESTAO AP_CODUNI     AP_AUTORIZ     AP_CMP  \
238   201408        MN    431490   2262584  4314203239313 2014-08-01   
485   201408        EP    430000   2251817  4314200108548 2014-08-01   
771   201408        EP    430000   2246929  4314204465769 2014-07-01   
970   201408        MN    431490   2262509  4314203320537 2014-08-01   
1255  201408        EP    430000   2246929  4314204465703 2014-06-01   
1307  201408        EP    430000   2242397  4314201717408 2014-08-01   
1485  201408        PG    432250   2241048  4314204400000 2014-08-01   
1578  201408        MN    431490   2262460  4314203316984 2014-08-01   
1587  201408        MN    431490   2237253  4314203316368 2014-08-01   
1633  201408        MN    431490   2262770  4314203239467 2014-08-01   

       AP_PRIPAL AP_VL_AP AP_UFMUN AP_TPUPS  ... fav_ACF_PREFAV  \
238   0305010107  1856.35   431490       36  ...              S   
485   0305010107    992.2   43078

In [20]:
print(df_merged[['fav_AP_CMP', 'ATD_DTPDR','meses_dialise', 'Cronico', 'fav_AP_MVM', 'fav_AP_CMP', 'fav_repeticao']].head(24))

   fav_AP_CMP  ATD_DTPDR  meses_dialise  Cronico fav_AP_MVM fav_AP_CMP  \
0  2016-11-01 2013-10-02           10.0  cronico     201611 2016-11-01   
1         NaT 2014-08-11            0.0    agudo        NaN        NaT   
2         NaT 2008-10-01           70.0  cronico        NaN        NaT   
3         NaT 2010-05-15           51.0  cronico        NaN        NaT   
4         NaT 2010-04-30           52.0  cronico        NaN        NaT   
5         NaT 2011-04-20           40.0  cronico        NaN        NaT   
6         NaT 2010-08-31           48.0  cronico        NaN        NaT   
7         NaT 2010-05-01           51.0  cronico        NaN        NaT   
8         NaT 2014-03-01            5.0  cronico        NaN        NaT   
9         NaT 2013-11-22            9.0  cronico        NaN        NaT   
10        NaT 2008-08-01           72.0  cronico        NaN        NaT   
11        NaT 2010-09-09           47.0  cronico        NaN        NaT   
12        NaT 2011-06-27           38.

## adicionando a espera da FAV (diferença de tempo entre primeira dialise(ATD_DTPDR) e a data de atendimento ao paciente(fav_AP_CMP))

In [21]:
# Verificação prévia dos dados
print("\nVerificação dos dados:")
print(f"Datas FAV válidas: {df_merged['fav_AP_CMP'].notna().sum()}")
print(f"Datas ATD_DTPDR válidas: {df_merged['ATD_DTPDR'].notna().sum()}")
print(f"Pacientes crônicos: {(df_merged['Cronico'] == 'cronico').sum()}")
print(f"FAVs com repetição 1: {(df_merged['fav_repeticao'] == 1).sum()}")

# Versão corrigida do cálculo
df_merged['espera_fav'] = df_merged.apply(
    lambda row: (row['fav_AP_CMP'] - row['ATD_DTPDR']).days 
    if (pd.notna(row['fav_AP_CMP']) and 
        pd.notna(row['ATD_DTPDR']) and 
        str(row['Cronico']).lower() == 'cronico' and 
        row['fav_repeticao'] == 1) 
    else None,
    axis=1
)

# Verificação pós-cálculo
print("\nResultado do cálculo:")
print(f"Valores não nulos em espera_fav: {df_merged['espera_fav'].notna().sum()}")
print(df_merged[df_merged['espera_fav'].notna()][['AP_CNSPCN', 'ATD_DTPDR', 'fav_AP_CMP', 'espera_fav']].head())


Verificação dos dados:
Datas FAV válidas: 432773
Datas ATD_DTPDR válidas: 762682
Pacientes crônicos: 28377
FAVs com repetição 1: 13374

Resultado do cálculo:
Valores não nulos em espera_fav: 2095
          AP_CNSPCN  ATD_DTPDR fav_AP_CMP  espera_fav
0   {}{|~}}} 2013-10-02 2016-11-01      1126.0
21  {{}{~~}}| 2013-09-15 2016-10-01      1112.0
23  {{||~~ 2013-03-27 2015-05-01       765.0
25  {{{{~} 2009-08-25 2020-05-01      3902.0
27  {{{} 2011-08-01 2016-02-01      1645.0


## criação do csv do merge de dialise e fav

In [22]:
# 7. Salvar resultado
df_merged.to_csv("dialise_fav_final4.csv", index=False, sep=";", encoding='utf-8')

In [23]:
print(df_merged.head())

   AP_MVM AP_CONDIC AP_GESTAO AP_CODUNI     AP_AUTORIZ     AP_CMP   AP_PRIPAL  \
0  201408        MN    431490   2237253  4314203219370 2014-08-01  0305010166   
1  201408        MN    431490   2262568  4314203319756 2014-08-01  0305010166   
2  201408        MN    431490   2262568  4314203240490 2014-08-01  0305010166   
3  201408        MN    431490   2237601  4314201305645 2014-08-01  0305010166   
4  201408        MN    431490   2237598  4314203319162 2014-08-01  0305010166   

  AP_VL_AP AP_UFMUN AP_TPUPS  ... fav_ACF_FLEBIT fav_ACF_HEMATO  \
0  2149.62   431490       05  ...              N              N   
1    400.0   431490       05  ...            NaN            NaN   
2  2149.62   431490       05  ...            NaN            NaN   
3  2204.49   431490       05  ...            NaN            NaN   
4  1529.46   431490       05  ...            NaN            NaN   

  fav_ACF_VEIAVI fav_ACF_PULSO fav_ACF_VEIDIA fav_ACF_ARTDIA fav_ACF_FREMIT  \
0              N             N 

In [24]:
# Verificar os dados relevantes
print(df_merged[['fav_AP_CMP', 'ATD_DTPDR', 'Cronico', 'fav_repeticao', 'espera_fav']].head(24))

   fav_AP_CMP  ATD_DTPDR  Cronico  fav_repeticao  espera_fav
0  2016-11-01 2013-10-02  cronico            1.0      1126.0
1         NaT 2014-08-11    agudo            0.0         NaN
2         NaT 2008-10-01  cronico            0.0         NaN
3         NaT 2010-05-15  cronico            0.0         NaN
4         NaT 2010-04-30  cronico            0.0         NaN
5         NaT 2011-04-20  cronico            0.0         NaN
6         NaT 2010-08-31  cronico            0.0         NaN
7         NaT 2010-05-01  cronico            0.0         NaN
8         NaT 2014-03-01  cronico            0.0         NaN
9         NaT 2013-11-22  cronico            0.0         NaN
10        NaT 2008-08-01  cronico            0.0         NaN
11        NaT 2010-09-09  cronico            0.0         NaN
12        NaT 2011-06-27  cronico            0.0         NaN
13        NaT 2009-03-10  cronico            0.0         NaN
14        NaT 2011-07-21  cronico            0.0         NaN
15        NaT 2013-04-23